## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [5]:
# Create a list of at least 10 column names to use as X data
features_list = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'HourlyRate', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[features_list]

# Show the data types for X_df
X_df.dtypes

Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
HourlyRate                 int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [8]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a OneHotEncoder for the Department column
dept_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_ohe = dept_ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_ohe_train = dept_ohe.transform(y_train[['Department']])
dept_ohe_test = dept_ohe.transform(y_train[['Department']])

In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_ohe = attrition_ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_ohe_train = attrition_ohe.transform(y_train[['Attrition']])
attrition_ohe_test = attrition_ohe.transform(y_train[['Attrition']])

## Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data
features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(features,), name='input_layer')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared_layer2')(shared_layer1)

In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(32, activation='relu', name='dept_hidden')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(3, activation='softmax', name='dept_output')(dept_hidden)

In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)

In [15]:
# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, dept_output])

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics={'dept_output': 'accuracy', 
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 64)        │        704 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 128)       │      8,320 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_hidden (Dense) │ (None, 32)        │      4,128 │ shared_layer2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         99 │ dept_hidden[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'attrition_output': attrition_ohe_train,
        'department_output': dept_ohe_train
    },
    epochs=100,  
    batch_size=32, 
    validation_split=0.2
)

Epoch 1/100


/opt/anaconda3/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - attrition_output_accuracy: 0.7427 - loss: 0.5506 - val_attrition_output_accuracy: 0.8281 - val_loss: 0.4990
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8635 - loss: 0.3742 - val_attrition_output_accuracy: 0.8326 - val_loss: 0.4677
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8401 - loss: 0.3894 - val_attrition_output_accuracy: 0.8326 - val_loss: 0.4817
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - attrition_output_accuracy: 0.8441 - loss: 0.3630 - val_attrition_output_accuracy: 0.8281 - val_loss: 0.4836
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - attrition_output_accuracy: 0.8686 - loss: 0.3226 - val_attrition_output_accuracy: 0.8190 - val_loss: 0.4818
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - attrition_output_accuracy: 0.8690 - loss: 0.3472 - val_attrition_output_accuracy: 0.8281 - val_loss: 0.4968
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0

In [17]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test_scaled), {
        'attrition_output': attrition_ohe_test, 
        'dept_output': dept_ohe_test
    })
results

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 368
'y' sizes: 1102, 1102


In [ ]:
# Print the accuracy for both department and attrition
print(f'Attrition Accuracy: {round((results[1] * 100), 2)}%\nDepartment Accuracy: {round((results[2] * 100), 2)}%')

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 